# Get the data

In [1]:
from get_notes import get_notes_from_files
import pickle
import os

In [2]:
DATA_PATH = '../../data/Mozart'
notes = []
notes_file_path = os.path.join(DATA_PATH, 'notes')

# If notes have already been dumped, load them, else run parser
if os.path.exists(notes_file_path):
    with open(notes_file_path, 'rb') as notes_file:
        notes = pickle.load(notes_file)
else:
    notes = get_notes_from_files(DATA_PATH)
    with open(notes_file_path, 'wb') as notes_file:
        pickle.dump(notes, notes_file)

Parsing 100 files from folder 'd:\Homework\4 Semester\MuseGen\MuseGen\data\Mozart'


100%|██████████| 100/100 [01:33<00:00,  1.07it/s]


# Set up the model

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from music21 import converter, note, chord
import numpy as np

### Define hyperparameters

In [4]:
SEQUENCE_LENGTH = 100
EPOCHS = 1  # Adjust as needed
BATCH_SIZE = 128  # Adjust as needed

### Prepare sequences for LSTM

In [5]:
# Create input sequences and corresponding output
unique_notes = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

input_sequences = []
output_sequences = []

for i in range(len(notes) - SEQUENCE_LENGTH):
    sequence_in = notes[i:i + SEQUENCE_LENGTH]
    sequence_out = notes[i + SEQUENCE_LENGTH]
    input_sequences.append([note_to_int[char] for char in sequence_in])
    output_sequences.append(note_to_int[sequence_out])

In [6]:
print(len(unique_notes))

5693


### Configure the model

In [12]:
# Reshape input sequences
x = np.reshape(input_sequences, (len(input_sequences), SEQUENCE_LENGTH, 1))
x = x / float(len(unique_notes))

# One-hot encode output sequences
y = to_categorical(output_sequences)

model = Sequential()
model.add(LSTM(512, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

check_point_path = "model/cp_{epoch:04d}.keras"
check_point = ModelCheckpoint(filepath=check_point_path, save_best_only=True, monitor='loss', save_freq='epoch')


MemoryError: Unable to allocate 20.0 GiB for an array with shape (470528, 5693) and data type float64

### Train the model

In [10]:
model.fit(x, y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[check_point])

  83/3676 ━━━━━━━━━━━━━━━━━━━━ 2:07:39 2s/step - loss: 8.3881

KeyboardInterrupt: 